### MLlib package of PySpark
스파크 2.0부터 ML 패키지는 데이터프레임에 대해 작동  
MLlist 패키지는 RDD에 대해서 작동함 

MLlib은 전체적으로 크게 세 단계의 머신러닝 기능으로 구분된다. 
- 데이터 전처리 : 피처 추출, 변형, 선택, 카테고리 피처에 대한 해석, 자연어 처리
- 머신러닝 알고리즘 : 몇몇 유명, 고급 레벨인 회귀, 분류, 군집 알고리즘 지원 
- 유틸리티 : 기술통계, 카이스퀘어(ChiSquare)테스트, 선행대수, 모델 평가

In [1]:
import findspark, pyspark
findspark.find()

from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession 
conf = pyspark.SparkConf().setAppName('appName').setMaster('local[2]')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

US 2014 ~ 2015 출생 데이터의 일부  
원본 데이터는 300개의 퓨처로 구성, 그 중 85개 선별  
799만개의 데이터 중 45429개 균등 샘플링  
INFANT_ALIVE_AT_REPORT 가 1인지 0인지 예측하는 것이 목표.  

In [3]:
import pyspark.sql.types as typ
labels = [
    ('INFANT_ALIVE_AT_REPORT', typ.StringType()),
    ('BIRTH_YEAR', typ.IntegerType()),
    ('BIRTH_MONTH', typ.IntegerType()),
    ('BIRTH_PLACE', typ.StringType()),
    ('MOTHER_AGE_YEARS', typ.IntegerType()),
    ('MOTHER_RACE_6CODE', typ.StringType()),
    ('MOTHER_EDUCATION', typ.StringType()),
    ('FATHER_COMBINED_AGE', typ.IntegerType()),
    ('FATHER_EDUCATION', typ.StringType()),
    ('MONTH_PRECARE_RECODE', typ.StringType()),
    ('CIG_BEFORE', typ.IntegerType()),
    ('CIG_1_TRI', typ.IntegerType()),
    ('CIG_2_TRI', typ.IntegerType()),
    ('CIG_3_TRI', typ.IntegerType()),
    ('MOTHER_HEIGHT_IN', typ.IntegerType()),
    ('MOTHER_BMI_RECODE', typ.IntegerType()),
    ('MOTHER_PRE_WEIGHT', typ.IntegerType()),
    ('MOTHER_DELIVERY_WEIGHT', typ.IntegerType()),
    ('MOTHER_WEIGHT_GAIN', typ.IntegerType()),
    ('DIABETES_PRE', typ.StringType()),
    ('DIABETES_GEST', typ.StringType()),
    ('HYP_TENS_PRE', typ.StringType()),
    ('HYP_TENS_GEST', typ.StringType()),
    ('PREV_BIRTH_PRETERM', typ.StringType()),
    ('NO_RISK', typ.StringType()),
    ('NO_INFECTIONS_REPORTED', typ.StringType()),
    ('LABOR_IND', typ.StringType()),
    ('LABOR_AUGM', typ.StringType()),
    ('STEROIDS', typ.StringType()),
    ('ANTIBIOTICS', typ.StringType()),
    ('ANESTHESIA', typ.StringType()),
    ('DELIV_METHOD_RECODE_COMB', typ.StringType()),
    ('ATTENDANT_BIRTH', typ.StringType()),
    ('APGAR_5', typ.IntegerType()),
    ('APGAR_5_RECODE', typ.StringType()),
    ('APGAR_10', typ.IntegerType()),
    ('APGAR_10_RECODE', typ.StringType()),
    ('INFANT_SEX', typ.StringType()),
    ('OBSTETRIC_GESTATION_WEEKS', typ.IntegerType()),
    ('INFANT_WEIGHT_GRAMS', typ.IntegerType()),
    ('INFANT_ASSIST_VENTI', typ.StringType()),
    ('INFANT_ASSIST_VENTI_6HRS', typ.StringType()),
    ('INFANT_NICU_ADMISSION', typ.StringType()),
    ('INFANT_SURFACANT', typ.StringType()),
    ('INFANT_ANTIBIOTICS', typ.StringType()),
    ('INFANT_SEIZURES', typ.StringType()),
    ('INFANT_NO_ABNORMALITIES', typ.StringType()),
    ('INFANT_ANCEPHALY', typ.StringType()),
    ('INFANT_MENINGOMYELOCELE', typ.StringType()),
    ('INFANT_LIMB_REDUCTION', typ.StringType()),
    ('INFANT_DOWN_SYNDROME', typ.StringType()),
    ('INFANT_SUSPECTED_CHROMOSOMAL_DISORDER', typ.StringType()),
    ('INFANT_NO_CONGENITAL_ANOMALIES_CHECKED', typ.StringType()),
    ('INFANT_BREASTFED', typ.StringType())
]

schema = typ.StructType([typ.StructField(e[0], e[1], False) for e in labels])

In [4]:
births = spark.read.csv('C:/Users/admin/Desktop/spark_data/data2/births_train.csv.gz', header=True, schema=schema)

In [5]:
type(births)

pyspark.sql.dataframe.DataFrame

In [6]:
births.printSchema()

root
 |-- INFANT_ALIVE_AT_REPORT: string (nullable = true)
 |-- BIRTH_YEAR: integer (nullable = true)
 |-- BIRTH_MONTH: integer (nullable = true)
 |-- BIRTH_PLACE: string (nullable = true)
 |-- MOTHER_AGE_YEARS: integer (nullable = true)
 |-- MOTHER_RACE_6CODE: string (nullable = true)
 |-- MOTHER_EDUCATION: string (nullable = true)
 |-- FATHER_COMBINED_AGE: integer (nullable = true)
 |-- FATHER_EDUCATION: string (nullable = true)
 |-- MONTH_PRECARE_RECODE: string (nullable = true)
 |-- CIG_BEFORE: integer (nullable = true)
 |-- CIG_1_TRI: integer (nullable = true)
 |-- CIG_2_TRI: integer (nullable = true)
 |-- CIG_3_TRI: integer (nullable = true)
 |-- MOTHER_HEIGHT_IN: integer (nullable = true)
 |-- MOTHER_BMI_RECODE: integer (nullable = true)
 |-- MOTHER_PRE_WEIGHT: integer (nullable = true)
 |-- MOTHER_DELIVERY_WEIGHT: integer (nullable = true)
 |-- MOTHER_WEIGHT_GAIN: integer (nullable = true)
 |-- DIABETES_PRE: string (nullable = true)
 |-- DIABETES_GEST: string (nullable = true)


In [7]:
# Y : yes
# N : no
# U : unknown
recode_dictionary = {
    'YNU' : {
        'Y' : 1, 
        'N' : 2, 
        'U' : 0 
    }
}

In [8]:
selected_features = [
    'INFANT_ALIVE_AT_REPORT', 
    'BIRTH_PLACE', 
    'MOTHER_AGE_YEARS', 
    'FATHER_COMBINED_AGE', 
    'CIG_BEFORE', 
    'CIG_1_TRI', 
    'CIG_2_TRI', 
    'CIG_3_TRI', 
    'MOTHER_HEIGHT_IN', 
    'MOTHER_PRE_WEIGHT', 
    'MOTHER_DELIVERY_WEIGHT', 
    'MOTHER_WEIGHT_GAIN', 
    'DIABETES_PRE', 
    'DIABETES_GEST', 
    'HYP_TENS_PRE', 
    'HYP_TENS_GEST', 
    'PREV_BIRTH_PRETERM'
]

births_trimmed = births.select(selected_features)

In [9]:
births_trimmed.head()

Row(INFANT_ALIVE_AT_REPORT='N', BIRTH_PLACE='1', MOTHER_AGE_YEARS=29, FATHER_COMBINED_AGE=99, CIG_BEFORE=99, CIG_1_TRI=99, CIG_2_TRI=99, CIG_3_TRI=99, MOTHER_HEIGHT_IN=99, MOTHER_PRE_WEIGHT=999, MOTHER_DELIVERY_WEIGHT=999, MOTHER_WEIGHT_GAIN=99, DIABETES_PRE='N', DIABETES_GEST='N', HYP_TENS_PRE='N', HYP_TENS_GEST='N', PREV_BIRTH_PRETERM='N')

데이터셋에는 Yes/No/Unknown 값을 가진 피처들이 매우 많음.   
Yes는 1, 나머지는 0으로 변환  

어머니의 흡연량 관련 레코드 :   
0 : 임신 기간 동안 금연   
1-97 : 한개 ~ 97개 사이   
98 : 98개 이상   
99 : 알 수 없음  

In [12]:
import pyspark.sql.functions as func 

def recode(col, key) :
    return recode_dictionary[key][col]

def correct_cig(feat) : 
    return func.when(func.col(feat) != 99, func.col(feat)) \
            .otherwise(0)
rec_integer = func.udf(recode, typ.IntegerType())

In [13]:
births_transformed = births_trimmed.withColumn('CIG_BEFORE', correct_cig('CIG_BEFORE')) \
                        .withColumn('CIG_1_TRI', correct_cig('CIG_1_TRI')) \
                        .withColumn('CIG_2_TRI', correct_cig('CIG_2_TRI')) \
                        .withColumn('CIG_3_TRI', correct_cig('CIG_3_TRI')) 

In [14]:
cols = [(col.name, col.dataType) for col in births_trimmed.schema]
cols

[('INFANT_ALIVE_AT_REPORT', StringType),
 ('BIRTH_PLACE', StringType),
 ('MOTHER_AGE_YEARS', IntegerType),
 ('FATHER_COMBINED_AGE', IntegerType),
 ('CIG_BEFORE', IntegerType),
 ('CIG_1_TRI', IntegerType),
 ('CIG_2_TRI', IntegerType),
 ('CIG_3_TRI', IntegerType),
 ('MOTHER_HEIGHT_IN', IntegerType),
 ('MOTHER_PRE_WEIGHT', IntegerType),
 ('MOTHER_DELIVERY_WEIGHT', IntegerType),
 ('MOTHER_WEIGHT_GAIN', IntegerType),
 ('DIABETES_PRE', StringType),
 ('DIABETES_GEST', StringType),
 ('HYP_TENS_PRE', StringType),
 ('HYP_TENS_GEST', StringType),
 ('PREV_BIRTH_PRETERM', StringType)]

In [ ]:
YNU_cols = []
for i, s in enumerate(cols) : 
    if s[1] == typ.StringType() : 
        dis = births.select(s[0]) \
            .distinct().rdd.map(lambda row : row[0]).collect()
        if 'Y' in dis : 
            YNU_cols.append(s[0])

In [ ]:
births.select('INFANT_ALIVE_AT_REPORT').distinct().rdd.map(lambda row : row[0]).collect()